In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
file_path='/content/drive/MyDrive/Kidney_segmentation_data/'

Mounted at /content/drive/


In [ ]:
import os
from glob import glob
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import albumentations as A
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import AveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Lambda
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import math
from keras.applications import VGG19
from keras.layers import GlobalAveragePooling2D 
from keras.layers import multiply, Reshape
from tensorflow.keras.optimizers import Nadam ,Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from tensorflow.keras.metrics import Recall, Precision
%matplotlib inline 

In [ ]:
# Enable mixed precision and tensor float 32 execution
tf.config.experimental.enable_mixed_precision = True
tf.config.experimental.enable_tensor_float_32_execution = True
tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [ ]:
print(physical_devices)
tf.config.run_functions_eagerly(True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
def Augmentation(images,mask):
    def _transform(image,mask):
        transform=A.Compose([
         A.HorizontalFlip(p=0.25),
         A.VerticalFlip(p=0.25),
         A.RandomRotate90(p=.5),
         A.RandomContrast(limit=0.2, p=0.1),
         A.Transpose(p=0.5),
         A.ElasticTransform(p=.4, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    ])
        aug=transform(image=image,mask=mask)
        img=aug['image']
        img=tf.cast(img,tf.float64)
        msk=aug['mask']
        msk=tf.cast(msk,tf.float64)
        return img,msk
    image,mask=tf.numpy_function(_transform,[images,mask],[tf.float64,tf.float64])
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    mask.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    return image,mask

In [ ]:
IMAGE_SIZE=256
AUTO = tf.data.experimental.AUTOTUNE
def load_data(split=0.3):
    images = sorted(glob(file_path+"train_256/*"))
    masks = sorted(glob(file_path+"masks_256/*"))
    total_size = len(images)
    valid_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=split, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split, random_state=42)

    valid_x, test_x = train_test_split(valid_x, test_size=split, random_state=42)
    valid_y, test_y = train_test_split(valid_y, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y),(test_x,test_y)
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)=load_data()
train_size=len(train_x)
valid_size=len(valid_x)



def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(256,256),interpolation=cv2.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float64)
    return x
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(256,256),interpolation=cv2.INTER_NEAREST)
    x = np.expand_dims(x, axis=-1)
    x = np.concatenate([x,x],axis=-1)
    x=tf.cast(x,dtype=tf.float64)
    return x

def parser(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    return x,y
    
def tf_dataset(x, y, batch,repeat_original=1):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(parser)
    original_dataset = dataset.repeat(repeat_original)

    augmented_dataset = dataset.map(Augmentation)
    dataset = tf.data.Dataset.concatenate(original_dataset, augmented_dataset)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(128,seed=0)
    dataset = dataset.batch(batch)
    return dataset
train_dataset=tf_dataset(train_x,train_y,batch=8)
valid_dataset=tf_dataset(valid_x,valid_y,batch=8)

IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_CHANNELS=3

In [ ]:
#LOSS FUNCTIONS 
from tensorflow.keras.losses import binary_crossentropy
smooth = 1
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)


In [ ]:
#models
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = multiply([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = [inputs]

    model = tf.keras.applications.DenseNet169(include_top=False, weights='imagenet',input_tensor=inputs)
    for layers in model.layers:
        layers.trainable=False
        
    #model.summary()
    names = ["conv1/relu", "pool2_conv", "pool3_conv", "pool4_conv"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("relu").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [ 256, 128, 64,64,32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_connections[i]])
        x = conv_block(x, f)

    return x
def encoder2(inputs):
    num_filters = [32,64,64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 64,32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x


def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = concatenate([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = concatenate([outputs1, outputs2])
    model = Model(inputs, outputs)
    return model


In [ ]:
model=tf.keras.models.load_model(file_path+"weights/"+"eighth.hd5",custom_objects={
        'dice_loss': dice_loss ,'dice_coef': dice_coef})

In [ ]:
# model=build_model((256,256,3))

device = '/GPU:0'
save_filepath= file_path+"weights/"+"ninth_model.hd5"
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(save_filepath,verbose=1,monitor='val_dice_coef',mode='max',save_best_only=True),
    CSVLogger(file_path+"dataDensenet161freeze.csv")
    ]
with tf.device(device):
    model.compile(loss=dice_loss,optimizer=Adam(learning_rate=0.00025),metrics=[dice_coef,Recall(),Precision()])
    t_steps=train_size//8
    v_steps=valid_size//8
with tf.device(device):
    history=model.fit(train_dataset, 
                            epochs =20,
                            steps_per_epoch = t_steps,
                            validation_data = valid_dataset,
                            validation_steps = v_steps,
                            callbacks=callbacks)

Epoch 1/20
822/838 [============================>.] - ETA: 54s - loss: 0.5523 - dice_coef: 0.4477 - recall: 0.8279 - precision: 0.5470

/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


838/838 [==============================] - ETA: 0s - loss: 0.5474 - dice_coef: 0.4526 - recall: 0.8260 - precision: 0.5503
Epoch 1: val_dice_coef improved from -inf to 0.71423, saving model to /content/drive/MyDrive/Kidney_segmentation_data/weights/ninth_model.hd5


838/838 [==============================] - 3898s 4s/step - loss: 0.5474 - dice_coef: 0.4526 - recall: 0.8260 - precision: 0.5503 - val_loss: 0.2858 - val_dice_coef: 0.7142 - val_recall: 0.8376 - val_precision: 0.8615
Epoch 2/20


/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


...

838/838 [==============================] - ETA: 0s - loss: 0.1222 - dice_coef: 0.8778 - recall: 0.8930 - precision: 0.9281
Epoch 17: val_dice_coef improved from 0.86704 to 0.86821, saving model to /content/drive/MyDrive/Kidney_segmentation_data/weights/ninth_model.hd5


838/838 [==============================] - 544s 649ms/step - loss: 0.1222 - dice_coef: 0.8778 - recall: 0.8930 - precision: 0.9281 - val_loss: 0.1318 - val_dice_coef: 0.8682 - val_recall: 0.8898 - val_precision: 0.9184
Epoch 18/20


/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1639: FutureWarning: RandomContrast has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


134/838 [===>..........................] - ETA: 6:33 - loss: 0.1810 - dice_coef: 0.8190 - recall: 0.8392 - precision: 0.8908

KeyboardInterrupt: ignored